In [1]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"
from google import genai
from google.genai import types

genai.__version__
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.6 MB/s eta 0:00:00


**Storing Information in a DB:**

In [2]:
%load_ext sql
%sql sqlite:///gym.db

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    goal VARCHAR(255) NOT NULL
);

CREATE TABLE IF NOT EXISTS user_preferences (
    preference_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    unit_system VARCHAR(50) NOT NULL, -- e.g., Metric, Imperial
    reminder_time INTEGER, -- in seconds before workout
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS workouts (
    workout_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    note VARCHAR(255) NOT NULL,
    date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    duration INTEGER, -- in minutes
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS exercises_lookup (
    exercise_id INTEGER PRIMARY KEY AUTOINCREMENT,
    exercise_name VARCHAR(255) NOT NULL UNIQUE,
    muscle_group VARCHAR(255) NOT NULL
);

CREATE TABLE IF NOT EXISTS exercises (
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    workout_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    sets INTEGER NOT NULL,
    weight INTEGER NOT NULL,
    reps INTEGER NOT NULL,
    FOREIGN KEY (workout_id) REFERENCES workouts(workout_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS progress (
    progress_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    weight_lifted INTEGER NOT NULL,
    reps_completed INTEGER NOT NULL,
    sets_completed INTEGER NOT NULL,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS goals (
    goal_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    goal_description TEXT NOT NULL,
    target_date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    achieved INTEGER DEFAULT 0, -- 0 = False, 1 = True
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

CREATE TABLE IF NOT EXISTS routines (
    routine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    routine_name VARCHAR(255) NOT NULL,
    description TEXT NOT NULL,
    difficulty_level VARCHAR(50) NOT NULL -- e.g., Beginner, Intermediate, Advanced
);

CREATE TABLE IF NOT EXISTS routine_exercises (
    routine_exercise_id INTEGER PRIMARY KEY AUTOINCREMENT,
    routine_id INTEGER NOT NULL,
    exercise_id INTEGER NOT NULL,
    sets INTEGER NOT NULL,
    reps INTEGER NOT NULL,
    rest_time_seconds INTEGER NOT NULL, -- in seconds
    FOREIGN KEY (routine_id) REFERENCES routines(routine_id),
    FOREIGN KEY (exercise_id) REFERENCES exercises_lookup(exercise_id)
);

CREATE TABLE IF NOT EXISTS user_routines (
    user_routine_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    routine_id INTEGER NOT NULL,
    start_date TEXT NOT NULL, -- ISO 8601 format (YYYY-MM-DD)
    end_date TEXT, -- ISO 8601 format (YYYY-MM-DD)
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (routine_id) REFERENCES routines(routine_id)
);

 * sqlite:///gym.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

**Answering Basic Gym Questions:**

In [4]:
introduction = """
🤖 Welcome to Your Personal Gym Helper! 💪

I’m here to guide you safely and effectively on your fitness journey. Whether you’re just starting out or looking to refine your routine, I’ll provide clear, actionable advice tailored to your goals.

Safety is my top priority—always listen to your body and consult a professional if needed.

Let’s get started! 😊

What’s your primary fitness goal? (e.g., lose weight, build muscle, improve endurance)
"""

In [5]:
import sqlite3
db_file = "gym.db"
db_conn = sqlite3.connect(db_file)

def list_tables() -> list[str]:
    print(' DB CALL:list_tables()')
    cursor=db_conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables=cursor.fetchall()
    return [t[0] for t in tables]

def describe_table(table_name:str) -> list[tuple[str,str]]:
    print(f'-DB CALL: describe_table({table_name})')
    cursor=db_conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema=cursor.fetchall()
    return [(col[1],col[2]) for col in schema]

def execute_query(sql:str) -> list[list[str]]:
    print(f' -DB CALL: execute_query({sql})')
    cursor=db_conn.cursor()
    cursor.execute(sql)
    return cursor.fetchall()

In [6]:
db_tools = [list_tables, describe_table, execute_query]

instruction = """
INSTRUCTIONS:
1. (You are a helpful chatbot that can interact with an SQL database as a personal gym helper.)
2. (You will take the users' questions related to their fitness journey. If the request involves retrieving information about their goals, past workouts, or stored preferences, you will turn it into an SQL query using the tools available. If the request involves logging new workout data, you will inform the user that this functionality is currently under development. Once you have the information needed for retrieval, you will answer the user's question using the data returned in a clear and actionable way. For requests that go beyond direct database interaction (like detailed exercise explanations, form tips, motivational messages, general nutrition advice, or exercise variations), you will provide helpful information based on your general fitness knowledge while acknowledging that this detailed information isn't directly retrieved from the database.)
3. (You will filter out non-ethical, NSFW content, and non-English content. You will respond with a statement saying that the message is non-appropriate.)
4. (You are a personal gym helper, so you will act professionally, provide safe and effective guidance, and respond in professional English.)
5. (When formulating SQL queries, focus on retrieving information relevant to the user's fitness goals, workout history, preferences, and available routines.)
6. (Prioritize using the `describe_table` tool to understand the schema of relevant tables before executing queries, especially when the user's request is specific.)
7. (Present the data retrieved from the database in a user-friendly format within your response.)
8. (If the user asks for recommendations (e.g., exercises, routines), you can use the database to provide suggestions based on their goals and potentially difficulty levels, if available. For more detailed information about the recommended items, you will provide general knowledge.)
9. (If a returning user is identified (based on their name), you will automatically retrieve and mention their current fitness goal. You will avoid asking for their name and goal again unless they indicate a change.)
10. (If the user asks to log a new workout, respond professionally stating: "Thank you for wanting to log your workout! This feature is currently under development. For now, I can help you retrieve information about your past workouts and goals.")

START WITH:
1. {introduction}
🤖 Welcome to Your Personal Gym Helper! 💪

I’m here to guide you safely and effectively on your fitness journey. Whether you’re just starting out or looking to refine your routine, I’ll provide clear, actionable advice tailored to your goals.

Safety is my top priority—always listen to your body and consult a professional if needed.

Let’s get started! 😊

What’s your primary fitness goal? (e.g., lose weight, build muscle, improve endurance)
2. If they indicate they have used this before, ask them for their:
    a. First name
    b. Last name
3. After receiving their first and last name (for returning users), you will:
    a. Use `execute_query` on the `users` table to find their record based on the provided first and last name.
    b. If a record is found, retrieve their `goal` from the database and respond with something like: "Welcome back, [First Name]! I see your current goal is [Goal]. How can I assist you today?"
    c. If no record is found, proceed as a new user and ask for their primary fitness goal.
4. After understanding their goal (new user) or confirming it (returning user), ask a follow-up question to gather more context, such as:
    a. "What type of workouts do you enjoy or have done in the past?"
    b. "Do you have access to a gym or do you prefer home workouts?"
    c. "How many days per week are you planning to work out?"
    d. "Do you have any specific exercises you are interested in?"
5. Based on their initial responses, you can then utilize the database tools to answer their questions or provide relevant information. For example:
    a. If they ask for exercises for a specific muscle group, use `describe_table` on `exercises_lookup` and then `execute_query` to find exercises. You can then provide general explanations and form tips for those exercises based on your knowledge.
    b. If they ask about available routines, use `list_tables` to check for `routines` and `routine_exercises`, then use `describe_table` and `execute_query` to provide details. You can offer general information about the benefits of the routine.
    c. If a returning user asks about their past workouts, use `describe_table` on `workouts` and `exercises`, then `execute_query` to retrieve that information.
    d. If the user asks to see their current goal, use `execute_query` on the `users` table to retrieve their `goal` based on their name.
    e. If the user asks a general fitness or nutrition question, provide a helpful and professional answer based on your general knowledge.
    f. If the user asks for variations on an exercise, provide some common and safe variations based on your knowledge.
    g. You can incorporate motivational messages periodically or in response to certain queries to encourage the user.
"""

client = genai.Client(api_key=GOOGLE_API_KEY)

chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)
response = chat.send_message("hi")
print(f"\n{response.text}")


🤖 Welcome to Your Personal Gym Helper! 💪

I’m here to guide you safely and effectively on your fitness journey. Whether you’re just starting out or looking to refine your routine, I’ll provide clear, actionable advice tailored to your goals.

Safety is my top priority—always listen to your body and consult a professional if needed.

Let’s get started! 😊

What’s your primary fitness goal? (e.g., lose weight, build muscle, improve endurance)

